In [2]:
import pandas as pd
import numpy as np
import glob
from PIL import Image
from scipy.stats import randint
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

# 1. Data

In [3]:
X_tr = []
y_tr = []
X_ts = []
y_ts = []

classnum = range(1032)
for i in classnum :
    file_train_names = '{}/{}/*.png'.format("syllable_train", i)
    for image in glob.glob(file_train_names) :
        train_image = Image.open(image)
        X_tr.append(np.array(train_image))
        y_tr.append(i)
        
    file_inf_names = '{}/{}/*.png'.format("syllable_inference", i)
    for image in glob.glob(file_inf_names) :
        ts_image = Image.open(image)
        X_ts.append(np.array(ts_image))
        y_ts.append(i)


In [4]:
# 새로운 축을 따라 배열 쌓기 (동일한 shape)
X_tr = np.stack(X_tr)
X_ts = np.stack(X_ts)

y_tr = np.array(y_tr)
y_ts = np.array(y_ts)

In [5]:
# vectorization/flat operation (to make the 2D input into 1D vector)
X_tr_flat = X_tr.reshape(X_tr.shape[0],-1)
X_ts_flat = X_ts.reshape(X_ts.shape[0],-1)

In [6]:
# PCA
pca = PCA()
pca.fit(X_tr_flat)

PCA()

In [7]:
# 변환
X_pc_tr = pca.transform(X_tr_flat)
X_pc_ts = pca.transform(X_ts_flat)

In [8]:
n_pc = sum(pca.explained_variance_ratio_.cumsum() < 0.7) #48

X_pc_tr = X_pc_tr[:,:n_pc]
X_pc_ts = X_pc_ts[:,:n_pc]

In [9]:
print(X_tr_flat.shape)
print(X_pc_tr.shape)

(41280, 3072)
(41280, 48)


# 2. Functions

In [10]:
# RandomSearch

from sklearn.model_selection import RandomizedSearchCV

def hypertuning(model, params, iters, X, y):
    rscv_search = RandomizedSearchCV(model, param_distributions = params, n_iter = iters, cv = 5, random_state = 4242)
    rscv_search.fit(X, y)
    DT_params = rscv_search.best_params_
    DT_score = rscv_search.best_score_
    return DT_params, DT_score

# 3. Models

## 1) Decision trees

### (1) Untuned

In [11]:
from sklearn import tree

DT_untuned = tree.DecisionTreeClassifier(random_state = 4242)
DT_untuned = DT_untuned.fit(X_pc_tr, y_tr)

In [38]:
score_DT_untuned_tr = DT_untuned.score(X_pc_tr, y_tr)
score_DT_untuned_val = cross_val_score(DT_untuned, X_pc_tr, y_tr, cv = 5).mean()
score_DT_untuned_ts = DT_untuned.score(X_pc_ts, y_ts)

### (2) Tuned

In [44]:
# 기본 hyperparameter 추출

print('DecisionTreeClassifier 기본 하이퍼 파라미터:\n', DT_untuned.get_params())

DecisionTreeClassifier 기본 하이퍼 파라미터:
 {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 4242, 'splitter': 'best'}


In [40]:
DT_params = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : randint(40, 80), 
    'min_samples_split' : randint(2, 10)
}

In [41]:
DT_tuned = tree.DecisionTreeClassifier()
DT_tuned_result = hypertuning(DT_tuned, DT_params, 10, X_tr_flat, y_tr)

DT_tuned.set_params(**DT_tuned_result[0])

score_DT_tuned_tr = DT_tuned.score(X_pc_tr, y_tr)
score_DT_tuned_val = DT_tuned_result[1]
score_DT_tuned_ts = DT_tuned.score(X_pc_ts, y_ts)

### (3) Visualization

In [34]:
from sklearn.tree import export_graphviz
import graphviz

In [36]:
tree_graph = export_graphviz(DT_untuned,   
                             out_file = "tree.dot", max_depth = 2,
                           feature_names = ['PC'+str(i+1) for i in range(X_pc_tr.shape[1])],  
                           class_names = ['Y'+str(i) for i in range(1032)],  
                           filled = True,           
                           rounded = True,          
                           special_characters = True)

## 2) Logistic regression classifier

### (1) Untuned

In [ ]:
from sklearn.linear_model import LogisticRegression

LR_untuned = LogisticRegression(random_state = 4242)
LR_untuned = LR_untuned.fit(X_pc_tr, y_tr)

In [ ]:
score_LR_untuned_tr = LR_untuned.score(X_pc_tr, y_tr)
score_LR_untuned_val = cross_val_score(LR_untuned, X_pc_tr, y_tr, cv = 5).mean()
score_LR_untuned_ts = LR_untuned.score(X_pc_ts, y_ts)

### (2) Tuned

In [ ]:
# 기본 hyperparameter 추출

print('LogisticRegression 기본 하이퍼 파라미터:\n', LR_untuned.get_params())

In [ ]:
LR_params = {
    'penalty' : ['none', 'l1', 'l2', 'elasticnet']
}

In [ ]:
LR_tuned = LogisticRegression(random_state = 4242)

LR_tuned_result = hypertuning(LR_tuned, LR_params, 10, X_pc_tr, y_tr)

LR_tuned.set_params(**LR_tuned_result[0])

score_LR_tuned_tr = LR_tuned.score(X_pc_tr, y_tr)
score_LR_tuned_val = LR_tuned_result[1]
score_LR_tuned_ts = LR_tuned.score(X_pc_ts, y_ts)

## 3) SVM

### (1) Untuned

In [ ]:
from sklearn.svm import SVC

SVM_untuned = SVC(random_state = 4242)
SVM_untuned = SVM_untuned.fit(X_pc_tr, y_tr)

In [ ]:
score_SVM_untuned_tr = SVM_untuned.score(X_pc_tr, y_tr)
score_SVM_untuned_val = cross_val_score(SVM_untuned, X_pc_tr, y_tr, cv = 5).mean()
score_SVM_untuned_ts = SVM_untuned.score(X_pc_ts, y_ts)

### (2) Tuned

In [ ]:
SVM_params = {
    'C' : randint(3, 10)
}

In [ ]:
SVM_tuned = SVC(random_state = 4242)

SVM_tuned_result = hypertuning(SVM_tuned, SVM_params, 10, X_pc_tr, y_tr)

SVM_tuned.set_params(**SVM_tuned_result[0])

score_SVM_tuned_tr = SVM_tuned.score(X_pc_tr, y_tr)
score_SVM_tuned_val = SVM_tuned_result[1]
score_SVM_tuned_ts = SVM_tuned.score(X_pc_ts, y_ts)

## 4) MLP

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader 

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
n_total = X_tr.shape[0]

n_train = int(n_total*0.8)

idx_total = np.arange(n_total)
np.random.shuffle(idx_total)

idx_train = idx_total[:n_train]
idx_val = idx_total[n_train:]

In [9]:
# tensor로 변환

X_tr_mlp = torch.tensor(X_pc_tr[idx_train])
X_val_mlp = torch.tensor(X_pc_tr[idx_val])
X_ts_mlp = torch.tensor(X_pc_ts)

X_tr_cnn = torch.tensor(X_tr[idx_train]/255)
X_val_cnn = torch.tensor(X_tr[idx_val]/255)
X_ts_cnn = torch.tensor(X_ts)

y_tr_dl = torch.tensor(y_tr[idx_train])
y_val_dl = torch.tensor(y_tr[idx_val])
y_ts_dl = torch.tensor(y_ts)

In [20]:
# dataset 만들기 (TensorDataset)

train_data_mlp = TensorDataset(X_tr_mlp, y_tr_dl)
val_data_mlp = TensorDataset(X_val_mlp, y_val_dl)
test_data_mlp = TensorDataset(X_ts_mlp, y_ts_dl)

train_data_cnn = TensorDataset(torch.permute(X_tr_cnn,(0,3,1,2)), y_tr_dl)
val_data_cnn = TensorDataset(torch.permute(X_val_cnn,(0,3,1,2)), y_val_dl)
test_data_cnn = TensorDataset(torch.permute(X_ts_cnn,(0,3,1,2)), y_ts_dl)

In [21]:
# loader 만들기

batch_size = 64

train_loader_mlp = DataLoader(train_data_mlp, batch_size = batch_size, shuffle = True, drop_last = True)
val_loader_mlp = DataLoader(val_data_mlp, batch_size = batch_size, shuffle = True, drop_last = True)
test_loader_mlp = DataLoader(test_data_mlp, batch_size = batch_size, shuffle = False, drop_last = False)

train_loader_cnn = DataLoader(train_data_cnn, batch_size = batch_size, shuffle = True, drop_last = True)
val_loader_cnn = DataLoader(val_data_cnn, batch_size = batch_size, shuffle = True, drop_last = True)
test_loader_cnn = DataLoader(test_data_cnn, batch_size = batch_size, shuffle = False, drop_last = False)

In [16]:
# MLP 만들기

class mlp_classifier(nn.Module):
    def __init__(self, in_dim, out_dim, hid_dim):
        super(mlp_classifier, self).__init__()
        self.layers = nn.Sequential(*[
            nn.Linear(in_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, out_dim)
        ])
    
    def forward(self, x):
        y_pred = self.layers(x)
        return y_pred

In [17]:
mlp_model = mlp_classifier(in_dim = X_tr_mlp.shape[1], out_dim = 1032, hid_dim = 32).to(device)

In [19]:
loss_fn = nn.CrossEntropyLoss().to(device)
opt = torch.optim.Adam(mlp_model.parameters(), lr = 0.001)

In [ ]:
# Train code

TRAIN_ACCURACY = []
TRAIN_LOSS = []
VAL_ACCURACY = []
VAL_LOSS = []

num_epochs = 100

for epoch in range(num_epochs):
    train_acc = 0
    train_loss = 0
    
    #====================================================#
    for data in train_loader_mlp:
        x = data[0].type(torch.FloatTensor).to(device)
        y_true = data[1].type(torch.LongTensor).to(device)
        opt.zero_grad()
        y_pred = mlp_model(x)
        y_pred_class = torch.argmax(y_pred, 1) # 행단위 최댓값
        loss = loss_fn(y_pred, y_true.flatten())
        acc = (y_pred_class == y_true).float().mean()
        loss.backward()
        opt.step()

        train_acc += acc
        train_loss += loss

    train_acc = train_acc/len(train_loader_mlp)
    train_loss = train_loss/len(train_loader_mlp)
    TRAIN_ACCURACY.append(train_acc)
    TRAIN_LOSS.append(train_loss)
    #====================================================#

    val_acc = 0
    val_loss = 0

    for data in val_loader_mlp:
        x = data[0].type(torch.FloatTensor).to(device)
        y_true = data[1].type(torch.LongTensor).to(device)
        y_pred = mlp_model(x)
        y_pred_class = torch.argmax(y_pred, 1)
        loss = loss_fn(y_pred, y_true.flatten())
        acc = (y_pred_class == y_true).float().mean()
        

        val_acc += acc
        val_loss += loss

    val_acc = val_acc/len(val_loader_mlp)
    val_loss = val_loss/len(val_loader_mlp)
    VAL_ACCURACY.append(val_acc)
    VAL_LOSS.append(val_loss)
    #====================================================#

    if (epoch+1)%10 == 0:
        print('[EPOCH {}] Tr Acc : {}, Tr Loss : {}, Val Acc : {}, Val Loss : {}'.format(epoch+1,100*train_acc, train_loss, 100*val_acc, val_loss))


## 5) CNN

In [23]:
class cnn_classifier(nn.Module):
    def __init__(self, in_ch, mid_ch, num_class = 1032):
        super(cnn_classifier, self).__init__()
        self.cnn_layers = nn.Sequential(*[
            nn.Conv2d(in_ch, mid_ch, kernel_size = 3, stride=1, padding = 'same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(mid_ch, mid_ch, kernel_size = 3, stride=1, padding = 'same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(mid_ch, mid_ch, kernel_size = 3, stride=1, padding = 'same'),
        ])
        self.fc_layer = nn.Linear(2048, num_class)
    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0),-1) # flatten ( reshape )
        y_pred = self.fc_layer(x)
        return y_pred

In [24]:
cnn_model = cnn_classifier(in_ch = 3, mid_ch = 32, num_class = 1032).to(device)

In [25]:
loss_fn = nn.CrossEntropyLoss().to(device)
opt = torch.optim.Adam(cnn_model.parameters(), lr = 0.001)

In [26]:
TRAIN_ACCURACY = []
TRAIN_LOSS = []
VAL_ACCURACY = []
VAL_LOSS = []

num_epochs = 30

for epoch in range(num_epochs):
    train_acc = 0
    train_loss = 0
    
    #====================================================#
    for data in train_loader_cnn:
        x = data[0].type(torch.FloatTensor).to(device)
        y_true = data[1].type(torch.LongTensor).to(device)
        opt.zero_grad()
        y_pred = cnn_model(x)
        y_pred_class = torch.argmax(y_pred, 1)
        loss = loss_fn(y_pred, y_true.flatten())
        acc = (y_pred_class == y_true).float().mean()
        loss.backward()
        opt.step()

        train_acc += acc
        train_loss += loss

    train_acc = train_acc/len(train_loader_cnn)
    train_loss = train_loss/len(train_loader_cnn)
    TRAIN_ACCURACY.append(train_acc)
    TRAIN_LOSS.append(train_loss)
    #====================================================#

    val_acc = 0
    val_loss = 0

    for data in val_loader_cnn:
        x = data[0].type(torch.FloatTensor).to(device)
        y_true = data[1].type(torch.LongTensor).to(device)
        y_pred = cnn_model(x)
        y_pred_class = torch.argmax(y_pred, 1)
        loss = loss_fn(y_pred, y_true.flatten())
        acc = (y_pred_class == y_true).float().mean()
        

        val_acc += acc
        val_loss += loss

    val_acc = val_acc/len(val_loader_cnn)
    val_loss = val_loss/len(val_loader_cnn)
    VAL_ACCURACY.append(val_acc)
    VAL_LOSS.append(val_loss)
    #====================================================#

    if (epoch+1)%3 == 0:
        print('[EPOCH {}] Tr Acc : {}, Tr Loss : {}, Val Acc : {}, Val Loss : {}'.format(epoch+1,train_acc, train_loss, val_acc, val_loss))
